# Lecture 2: Functions and procedural abstraction, FP

* Functions
* Some brief notes on style, debugging and testing.
* Declarative patterns and functional programming.
* Presentation by Anders Märak Leffler
* Attribution: extends work by Johan Falkenjack.
* License: [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/)

# Bonus information

* Early [developer preview of Python 3.9](https://www.python.org/downloads/release/python-390a3/) available.
* [Terry Jones in memoriam](https://www.bbc.co.uk/news/av/entertainment-arts-51215814/terry-jones-michael-palin-pays-tribute-to-monty-python-star) (watch after lecture, not mandatory)
* Labs

## Why procedural abstraction?

In [ ]:
# A totally made up script, that goes a little overboard (for demonstration purposes).

# First we read the data
with ... :
    # some lines
    
# Then remove all the duplicates
# ...

# Then extract the relevant parameters by some algorithm.

# cut and paste
# cut and paste  (interacts with values from above)
# cut and paste  (interacts with values from above)
# cut and paste  (interacts with values from above)


# Make sure that model_is_consistent is True iff the model is consistent (with some property).

# cut and paste



if model_is_consistent:
    # Lots of code for plotting.
    # (without _any_ procedural abstraction this would be insane!)
    

# First we read the data
with ... :
    # some lines
    
# Then remove all the duplicates
# ... (maybe bug? Check later)

# Then extract the relevant parameters by some algorithm.

# cut and paste
# cut and paste  (interacts with values from above)
# cut and paste  (interacts with values from above)
# cut and paste  (interacts with values from above)


# Make sure that model_is_consistent is True iff the model is consistent (with some property).

# cut and paste



if model_is_consistent:
    # Lots of code for plotting. 
    # (without _any_ procedural abstraction this would be insane!)

...and then something goes wrong. Why? Where?

In [1]:
# Example, cleaned up.

def run_simulation(fname):
    """Generate a model of data in file fname, and plot it if consistent."""
    data = read_data(fname) 
    remove_duplicates(data) 
    model = extracted_parameters(data)    # change this line to use other algorithm
    if model.is_consistent():
        plot(data, model)

def read_data(fname):
    """Return the data contents of file named fname."""
    pass

def remove_duplicates(dataset):
    """Remove duplicates from mutable input dataset, return in format X."""
    pass

def extracted_parameters(dataset):
    """Return a model based on the dataset, with features XYZ"""
    pass

In [2]:
help(read_data)

Help on function read_data in module __main__:

read_data(fname)
    Return the data contents of file named fname.



### Yes, but what does this encapsulation and structuring add?

Bad:
* Slight overhead. Consider if you call a small function many many times.

Good:
* Readability.
* Testing and debugging.
    * Something went wrong. Why? Code, data, an interaction?
    * Where did it go wrong?
    * Can we trust the individual parts? Test them?
    * Copy-paste issues.
* Code reuse
    * Do this again, with different data source.
    * Reuse parts of the program.
        * Note: subtle variable issues.
        
... Etcetera ...

Caveats:
* Sometimes having only a simple script makes sense.
* Sometimes replacing function calls with their code makes *enough* sense from a performance perspective. 

## Functions in Python

* Typically defined with `def` or a `lambda`.
* First-class (callable) values. *A function is just an object.*
    * Corollary: you can't have two distinct f(x) and f(x,y) [without tricks].
* Called by `function_name(<arguments>)`.

In [4]:
# hungarian
def hungarian_method():
    print("My hovercraft is full of eels.")

def print_args(x):
    print("I was called with: ", x)
    return x

In [6]:
hungarian_method()

My hovercraft is full of eels.


In [7]:
hungarian_method(123)

TypeError: hungarian_method() takes 0 positional arguments but 1 was given

In [8]:
# What happens here?
print_args(99)

I was called with:  99


99

In [9]:
# What happens here?
print_args(99) + 1000

I was called with:  99


1099

In [10]:
hungarian_method() + 1000

My hovercraft is full of eels.


TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [11]:
# What will this do? Crash? Print? Return?
print_args(hungarian_method)

I was called with:  <function hungarian_method at 0x7f2bd14401e0>


<function __main__.hungarian_method()>

In [12]:
hungarian_method

<function __main__.hungarian_method()>

In [13]:
import dis
dis.dis(hungarian_method)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('My hovercraft is full of eels.')
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [15]:
def run_arg(f):
    f()
    


In [17]:
run_arg(hungarian_method)

My hovercraft is full of eels.


In [18]:
run_arg(5)

TypeError: 'int' object is not callable

In [20]:
dir(hungarian_method)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [22]:
hm = hungarian_method

In [24]:
hm()

My hovercraft is full of eels.


In [25]:
hm is hungarian_method

True

In [26]:
hm.__name__

'hungarian_method'

In [29]:
retval = hungarian_method()
print(f"Return value was {retval}")

My hovercraft is full of eels.
Return value was None


* Lambdas contain a single expression, and have implicit return.

In [31]:
# Single argument lambda
sq = lambda x : x**2

# Two argument lambda
add = lambda x, y : x + y

# No argument lambda
return_five = lambda : 5

In [33]:
sq(5)

25

In [34]:
sq

<function __main__.<lambda>(x)>

In [35]:
# What can't you do with a lambda?
bad_lambda = lambda n : while n > 0: n = n - 1

SyntaxError: invalid syntax (<ipython-input-35-3b4071dca880>, line 2)

General idea: lambda [vars] : [expression that "becomes" a value when evaluated]

(Yes you can get around this, even without wrapping your code in a function, but you shouldn't. Use a `def` instead in that case.)

In [ ]:
# Optional task: figure out how to make a lambda that imports the math module, requests an input and 
# prints the square root of that input when run.

# This is really bad style, but might be fun to think about in order to understand the language. 
# This also suggests another reason as to why your code shouldn't eval(...) user input willy-nilly, as some textbooks
# suggest.

* **When are arguments evaluated?** Applicative-order evaluation.

In [37]:
# Argument evaluation. 

print_args(hungarian_method)

I was called with:  <function hungarian_method at 0x7f2bd14401e0>


<function __main__.hungarian_method()>

In [38]:
print_args( hungarian_method() )

My hovercraft is full of eels.
I was called with:  None


In [39]:
# What happens to unused parameters? Two-arg.
def test(arg, use_2nd_arg):
    if use_2nd_arg:
        print(arg)
    else:
        print("--- Didn't really need the value.")
        
test(hungarian_method(), False)  # Don't need the value actually.

My hovercraft is full of eels.
--- Didn't really need the value.


* Keyword arguments. Useful to clarify calls!

In [41]:
def identity(x):
    return x

In [44]:
test(use_2nd_arg=True, arg=identity(99))

99


You can combine keyword arguments and regular positional, but positional come first.

[Note: Core developer Raymond Hettinger argues for (over)using this in [PyCon 2013: Transforming Code into Beautiful, Idiomatic Python](https://youtu.be/OSGv2VnC0go?t=1841)]

* Default arguments. Should follow non-default arguments.

In [47]:
def test(use_2nd_arg, arg=5):
    if use_2nd_arg:
        print(arg)
    else:
        print("--- Didn't really need the value.")

In [46]:
# Try out calling this

test(True)

5


In [48]:
test(True, 123123123)

123123123


In [49]:
test(arg = 99)

TypeError: test() missing 1 required positional argument: 'use_2nd_arg'

Helps with understanding [the documentation](https://docs.python.org/3.7/library/functions.html#print).

In [50]:
# What will happen here?

def add_to_list(e, lst=[]):
    lst.append(e)
    return lst

my_list = add_to_list(999)
add_to_list(123, my_list)
print(my_list)

my_other_list = add_to_list("bat")
add_to_list("man", my_other_list)
print(my_other_list)

[999, 123]
[999, 123, 'bat', 'man']


In [51]:
# Pattern to counter this.

def add_to_list(e, lst=None):
    if lst is None:
        lst = []
    lst.append(e)
    return lst

In [52]:
del add_to_list

* Functions can be defined within functions.
* Static/lexical scoping. What will the following print?

In [53]:
# What will this print?

x = 5
def f():
    print("I'm in f! Now x is ", x)
    
def g(x):
    print("I'm in g! Now x is ", x)
    f()

print("--- Calling f.")
f()

print("--- Calling g (which in turn calls f).")
g(99)

--- Calling f.
I'm in f! Now x is  5
--- Calling g (which in turn calls f).
I'm in g! Now x is  99
I'm in f! Now x is  5


* LEGB lookup (first Local, then Enclosing, Global and finally Builtin scope).
* Binds values locally (unless told otherwise).

Useful resource: [www.pythontutor.com](https://www.pythontutor.com)

In [54]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

Before, x is 1
In f, x is 1
Afterwards, x is 1


In [55]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    x = 5
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

Before, x is 1
In f, x is 5
Afterwards, x is 1


In [56]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    x = x + 1
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

Before, x is 1


UnboundLocalError: local variable 'x' referenced before assignment

In [58]:
# What will happen?

x = 1
print("Before, x is", x)
def f():
    global x
    x = x + 1
    print("In f, x is", x)
    
f()
print("Afterwards, x is", x)

Before, x is 1
In f, x is 2
Afterwards, x is 2


In [ ]:
# Exercise: play around with the keywords nonlocal and global.

**Takeaway:** encapsulation!

   * Function + their scope/context (the latter word used losely) form a **closure** (or "lexical closure").

In [59]:
x = 1
y = 2

def f(x):
    def g():
        print("In this g, x is", x, " and y is ", y)
    return g

f(100)
h100 = f(100)
h55 = f(55)

In [60]:
h55()

In this g, x is 55  and y is  2


In [62]:
# Note: the lambda doesn't get x0 or x1 by parameter. It has to get them from somewhere!

def points_close_to(points, x0, y0, max_dist):
    is_near = lambda coord : (coord[0] - x0)**2 + (coord[1] - y0)**2 <= max_dist**2
    #return list(filter(is_near, points))
    return [p for p in points if is_near(p)]

points = [ (1,2), (5,9), (3,3), (2, 5) ]
points_close_to(points, 3,3, 3)

# Could the points be written in a more readable way? Stay tuned.

[(1, 2), (3, 3), (2, 5)]

In [63]:
# Bonus exercise (way outside the scope of this course): can we reach into a closure? To read values? Write?
# Python is flexible with certain things, and others not so much.

h100 = f(100)   
print(h100)
# Play around with h100. Check the dunder methods. (those that start and end with __)

<function f.<locals>.g at 0x7f2bd01f29d8>


[//]: # (Functions are just another type of value. Nothing special.)

# A note on naming, style, testing

---
**YOU DO NOT WRITE CODE FOR THE INTERPRETER. YOU WRITE IT FOR OTHER HUMAN READERS.**

---

![Abstruse Goose](https://abstrusegoose.com/strips/you_down_wit_OPC-yeah_you_know_me.png)

Attribution: [Abstruse Goose #432](https://www.abstrusegoose.com/432). CC BY-NC 3.0 US license.

**WHO ARE THOSE "OTHER PEOPLE"?**

Note: What good did the comment above do?

### Python function style interlude

> Names that are visible to the user as public parts of the API should follow conventions that reflect usage rather than implementation.

The "Overriding Principle" from the official [PEP 8 -- Style Guide for Python Code](https://www.python.org/dev/peps/pep-0008/#overriding-principle).

**In other words: focus on ** ***what*** **it does (or return), not** ***how*** **it does it.**

In [ ]:
# What does this do? Sample inputs?

def f(x):
    for i in x:
        x[i] += 1

In [73]:
def f(x):
    for i in x:
        print(f"Currently i is {i}")
        print(f"Currently x[i] is {x[i]}")
        x[i] = x[i] + 1

In [74]:
mylist = [1,2,3]
f(mylist)
print(mylist)

Currently i is 1
Currently x[i] is 2
Currently i is 3


IndexError: list index out of range

In [71]:
def f(mapping):
    for key in mapping:
        mapping[key] += 1
        
vals = {"a" : 1, "b" : 100}
f(vals)
print(vals)

Current i is a, and x is {'a': 1, 'b': 100}
Current i is b, and x is {'a': 2, 'b': 100}
{'a': 2, 'b': 101}


In [72]:
def increment_all(mapping):
    for key in mapping:
        mapping[key] += 1
        
vals = {"a" : 1, "b" : 100}
increment_all(vals)
print(vals)

{'a': 2, 'b': 101}


Consider
* function name.
* parameter names.
* variables.

**Not** required in labs, but useful. Official guidelines: [PEP 257 -- Docstring Conventions](https://www.python.org/dev/peps/pep-0257/).

* How would we document this?

In [75]:
def increment_all(mapping):
    """Increment all values in the mapping by 1."""
    for key in mapping:
        mapping[key] += 1

In [76]:
help(increment_all)

Help on function increment_all in module __main__:

increment_all(mapping)
    Increment all values in the mapping by 1.



### What do we require going forward?

* PEP8 compliance *not* required.
* Readable code.
* Considered variable names, iteration variables.
* Comments if we have larger code blocks. (Should this be needed?)
    * Why is this code here (what does it add), not what does it do ("first we set i to 0, then...").
    * You likely won't need a lot of this.
* Moderately clever structure. (Esp relevant in later labs.)

Good, but not required (unless we say so)
* Docstrings.

**DON'T PANIC**

## A few general hints for design and naming

Caveat: large subject, involving personal preferences and official guidelines (sometimes at company level). Trying to cover many beginner mistakes.

### Don't try to do everything

Struggling with naming can mean that a function does too many different things.

### Effects vs computation

Separate effects (print this, write file, change data by deaveraging...) from computation (is this valid data? What is the mean of this?).

### Taking the calling function's perspective

Compare    
    ```if calculate_if_data_consistent(data):
        ...```
        
with
    
```if is_consistent(data):
       ...```

### Don't mislead
`is_consistent(data)` should test if it is consistent, not change the data, print something or the like.

### Consider what the data represents

What is the application? Domain? Cf last lecture.

In [78]:
# Example
names = ["Alonzo", "Zeno", "Trisse"]

# Bad. j sounds like an index. (i,j,k...)
for i, j in enumerate(names):
    print(f"The {i}st/th name is {j}.")

The 0st/th name is Alonzo.
The 1st/th name is Zeno.
The 2st/th name is Trisse.


In [79]:
# OK, we know that it's a list of strings. But the strings _represent_ something.

names = ["Alonzo", "Zeno", "Trisse"]

for i, string in enumerate(names):
    print(f"The {i}st/th name is {string}.")

The 0st/th name is Alonzo.
The 1st/th name is Zeno.
The 2st/th name is Trisse.


In [80]:
# OK, we know that it's a list of strings. But the strings _represent_ something.

names = ["Alonzo", "Zeno", "Trisse"]

for i, name in enumerate(names):
    print(f"The {i}st/th name is {name}.")

The 0st/th name is Alonzo.
The 1st/th name is Zeno.
The 2st/th name is Trisse.


### Consider what your functions depend on

Keep in mind what outside variables your function depends on.

### Other design issue

* In *small-scale* top-down design, creating "empty" functions with a certain contract can be helpful.
    * Example: "In my program I will need a function `read_data` which takes a file name for a CSV file and returns a data set on format *X*." (this is the - somewhat handwaving - contract) "I'll define it, and fill it in later."
    * A Python function body cannot be empty. `pass` is your friend.

In [82]:
# How to write the read_data skeleton.

def read_data(filename, delimiter=","):
    """Return a ... with data from CSV file filename."""
    pass   # Do nothing instruction (NOP).

# continue coding.

## A note on testing and (unsystematic) debugging

> It has been just so in all of my inventions. The first step is an intuition, and comes with a burst, then difficulties arise - this thing gives out and [it is] then that 'Bugs' - as such little faults and difficulties are called - show themselves and months of intense watching, study and labor are requisite.

*Thomas Edison*, quoted in Ammann & Offutt (2017) - Intro to Software Testing, 2nd ed.

* Plenty of tools out there for debugging.
    * One example: [pdb](https://docs.python.org/3.7/library/pdb.html).
* We will focus on simpler print-style and asserts.

**IF YOU KNOW WHAT YOUR PROGRAM IS SUPPOSED TO DO, YOU CAN CHECK IF IT ACTUALLY DOES IT**

In [ ]:
# Faulty code. Somewhat articificial example. 
# Also an inefficient method, but let's not worry about that.

def calculate_if_prime_or_not(x):
    # Check to see if we can find something that divides the number.
    for y in range(x):
        if y % x == 0:
            answer = False
        else:
            answer = True
    return answer

What are the issues? Found how?

In [ ]:
# What is expected?
# Printing!
#...

Python Tutor may help here as well!

* We can add assertions to show that our (pure) function passed some useful tests. Corner cases to illustrate?

In [86]:
def successor(n):
    return n + 1

assert successor(0) == 1, "successor to first natural is 1"
assert not successor(0) == 999, "this is a problem"

print("We got here without problems!!")

We got here without problems!!


In [ ]:
# assert is_prime(???) == ???, "???"
assert True, "this should succeed"
#assert False, "this should fail"
assert not is_prime(4), "known non-prime 4 should not be considered prime"
assert is_prime(2), "the even number 2 should be considered prime"
#assert ...

# assert statement

# Add corner cases. 2 is the only non-odd prime. Empty lists...
# Cases both for expected success and failure.

print("All tests OK.")

[Note: We might have other forms of properties that might be suitable for automatic testing. For instance "all return values should be True/False", "all multiples of two integers should be non-prime" or the like. This is outside the  scope of this course, but an interesting subject to look into.

Modest focus here: strategies to find good test cases to convince ourselves somewhat. Links to books with coverage criteria in the lab.]



* We can gather this into functions.

In [ ]:
def test_is_prime():
    assert is_prime(2), "even number 2 should be considered prime"
    assert not is_prime(4), "known non-prime 4 should not be considered prime"
    # Could be written is_prime(4) == False, but this is ugly without increasing clarity.
    
    print("--- test_is_prime succeeded")
test_is_prime()

Note: primitive structure, but the logic can be adapted to proper frameworks.

## New type interlude: namedtuples

* Behave like tuples, but also allow member access by names. Eg mytuple.args instead of mytuple[0]. Enhanced readability.

In [91]:
from collections import namedtuple
# Define our coordinate type.
coord = namedtuple("coord", ["x", "y"])
p1 = coord(1,2)
p1.x

1

# Paradigms

* Philosophies about code.
    * Execution model.
    * Code organization.
* Python is a multi-paradigm language (sort of).

# Functional programming

* Typically declarative.
* Purity (see below) makes it easy to reason about. (Not necessarily write...)
    * In particular for parallel computation.
    * MapReduce.

### Pure functions

* No side causes. 
    * Doesn't depend on the state (of the world).

In [ ]:
import datetime

def has_side_causes():
    return datetime.datetime.now()

def no_side_causes():
    return 5 + 5
    
has_side_causes()

* No side effects.
    * Doesn't change the state of the world.

Corollary: In (purely) functional programming, 
* there will be less of hidden dependencies on state.
* reduced complexity.
* referential transparency. Calling function $f$ with input $X$ will always yield the same result.
* mathematical resoning useful (not just "do X, then Y").

## Feature: higher order functions

* Functions with functions as outputs are generally considered higher-order. Have you seen those before? (Functions + scope.)

In [ ]:
# Exercise.

* Similarly with functions taking functions as input, or returning functions as output.

In [ ]:
# Implementing our own filter, with a list comprehension.
points = [ coord(1,2), coord(-2,3), coord(5,100), coord(0,-1) ]

def points_satisfying(points, to_keep):  # to_keep behaves as a function
    """Return a list of points which satisfy the criterion set out by the to_keep function."""
    return [p for p in points if to_keep(p)]  # if we treat it like one

# How to call, to find points in the cone x >= 0, y >= 0?

## Declarative patterns

* Many declarative-style patterns are Pythonic.
* Often implemented with iterators.
* Useful to chain together, one result feeding into the next.
* Often well-implemented.
* Example: map.

In [ ]:
# map is a standard functional programming pattern.
# Cf mathematical mapping. map takes a sequence of values and produces the image of that sequence under a given function.

print(map(lambda x : x**2, [1,2,3]))
tuple(map(lambda x : x**2, [1,2,3]))

In [ ]:
# Caveat: in Python this produces something which generates the values as you ask for them.

squares = map(lambda x : x**2, [1,2,3])
print(squares)
print(list(squares))   # Consumes all the values.
print(tuple(squares))  # All consumed.

Corollary: this can be quite efficient. Doesn't generate values in vain.

* Example: filter

In [ ]:
# filter keeps those values which conform to a predicate (function which - ideally - returns True/False depending
# on if a criterion has been satisfied).

is_even = lambda n : n % 2 == 0       # could be written "not n % 2"
filter(is_even, [1,2,3])
list(filter(is_even, [1,2,3]))

* Example reduce. 

In [ ]:
from functools import reduce

reduce(lambda acc, e : [acc] + [e], [1,2,3, 4])   # [  [ [1,2] ] + [3]  ] + [4]

* These can be combined.

In [ ]:
sq = lambda x : x**2
seq = range(100)

square_evens = map(sq, filter(lambda n : n % 2 == 0, seq))
#list(square_evens)

## A note on recursion

* Practiced in lab 3. 
* Function values defined in terms... of function values.
* Focus on mathematical identities.
* Sometimes useful in its own right.
* Sometimes inefficient, but useful when turned into non-recursive algorithms.
* Use cases here: many tree-structures.

* Strategy
    * "Pretend" that we know the solution to a smaller case, extend that.
    * What do we know about the *smallest* case (or the smallest cases)?

In [ ]:
# What is the sum 0 + 1 + ... + (n-1) + n?

# Helper function (during program construction).
def sum_reference(n):
    """Return the sum 0+...+n."""
    return sum(range(n+1))

# If we knew the sum 0 + 1 + ... + (n-1)?

def sum_rec(n):
    pass

* Handwaving explanation above. Good mathematical foundations. 
    * Closely related to inductive proofs.
    * Well-orderings.

* Sanity checks:
    * base case (or base cases). 
    No more subdivision of the problem.
    Typical examples: empty list, zero, reached the leaves in a decision tree.
    * recursive case. Split into smaller case, and write the solution to this case. (Typically: rest of the list, traverse left or/and right branches of the tree...)